### Import required libraries:

In [1]:
import pandas as pd
import numpy as np

fire = pd.read_csv("C:\\Users\\hp\\Desktop\\Forestfire Project\\data\\FIRE DATA.csv")
ndvi = pd.read_csv("C:\\Users\\hp\\Desktop\\Forestfire Project\\data\\NDVI.csv")
dem = pd.read_csv("C:\\Users\\hp\\Desktop\\Forestfire Project\\data\\DEM.csv")
weather = pd.read_csv("C:\\Users\\hp\\Desktop\\Forestfire Project\\data\\WEATHER DATA.csv")

print(fire.head())

         date  latitude  longitude  brightness  confidence        frp
0  01-01-2023     22.63       88.5  380.936116          95   6.497069
1  16-01-2023     22.63       88.5  381.011340          88  28.004585
2  31-01-2023     22.63       88.5  386.707232          76  39.036593
3  15-02-2023     22.63       88.5  391.324055          76  35.424485
4  03-02-2023     22.63       88.5  351.134240          51   8.604461


In [2]:
fire = fire[['latitude','longitude','brightness','confidence','frp']]

In [3]:
fire.head()

,latitude,longitude,brightness,confidence,frp
0,22.63,88.5,380.936116,95,6.497069
1,22.63,88.5,381.011340,88,28.004585
2,22.63,88.5,386.707232,76,39.036593
3,22.63,88.5,391.324055,76,35.424485
4,22.63,88.5,351.134240,51,8.604461


In [4]:
weather.head()

,date,latitude,longitude,Temperature_Avg,Humidity,Rainfall,Wind_Speed
0,01-01-2023,22.63,88.5,10.076445,15.629449,19.443332,4.489457
1,16-01-2023,22.63,88.5,32.353542,60.482734,0.285540,11.040648
2,31-01-2023,22.63,88.5,14.943529,86.258974,12.026336,7.147528
3,15-02-2023,22.63,88.5,42.221985,53.278899,11.252652,19.319875
4,03-02-2023,22.63,88.5,13.277151,67.221367,2.694315,19.270992


In [5]:
weather = weather[['date',	'latitude',	'longitude','Temperature_Avg','Humidity','Rainfall','Wind_Speed']]

In [6]:
weather.head()

,date,latitude,longitude,Temperature_Avg,Humidity,Rainfall,Wind_Speed
0,01-01-2023,22.63,88.5,10.076445,15.629449,19.443332,4.489457
1,16-01-2023,22.63,88.5,32.353542,60.482734,0.285540,11.040648
2,31-01-2023,22.63,88.5,14.943529,86.258974,12.026336,7.147528
3,15-02-2023,22.63,88.5,42.221985,53.278899,11.252652,19.319875
4,03-02-2023,22.63,88.5,13.277151,67.221367,2.694315,19.270992


In [13]:
dem.head()

,latitude,longitude,elevation,slope,aspect
0,22.63,88.50,1186.166345,7.175655,122.783886
1,25.38,83.02,2857.071489,42.794687,40.850468
2,30.87,76.71,2222.782431,45.647103,332.889703
3,26.84,81.07,1836.109604,33.676632,315.842167
4,30.70,76.52,552.454057,46.258031,92.858986


In [15]:

ndvi.head()

,date,latitude,longitude,NDVI,soil_moisture,temperature,rainfall
0,01-01-2023,22.63,88.5,0.060190,46.119353,9.884229,1.662354
1,16-01-2023,22.63,88.5,0.213957,37.542525,13.967073,8.446302
2,31-01-2023,22.63,88.5,0.403306,24.926279,13.590147,3.862833
3,15-02-2023,22.63,88.5,0.418187,24.114157,12.343355,16.623542
4,03-02-2023,22.63,88.5,0.375138,27.420927,11.007707,9.496210


### Scientific Pipeline

In [18]:
import pandas as pd
import numpy as np

# Merge NDVI and DEM (real geographic grid)
grid = ndvi.merge(dem, on=['latitude','longitude'], how='inner')

# Attach weather to each location
weather_sample = weather.sample(
    n=len(grid),
    replace=True,
    random_state=42
)

grid = pd.concat(
    [grid.reset_index(drop=True),
     weather_sample.reset_index(drop=True)],
    axis=1
)

print("Grid created:", grid.shape)

Grid created: (1625, 17)


In [20]:
grid['veg_dryness'] = 1 - grid['NDVI']
grid['temp_risk'] = grid['Temperature_Avg'] / 50
grid['humidity_risk'] = 1 - (grid['Humidity'] / 100)
grid['wind_risk'] = grid['Wind_Speed'] / 50
grid['rain_risk'] = 1 - (grid['Rainfall'] / 50)
grid['terrain_risk'] = (
    grid['slope'] / grid['slope'].max()
)

In [22]:
grid['fire_risk_score'] = (
    0.30 * grid['veg_dryness'] +
    0.25 * grid['temp_risk'] +
    0.15 * grid['humidity_risk'] +
    0.15 * grid['wind_risk'] +
    0.10 * grid['rain_risk'] +
    0.05 * grid['terrain_risk']
)

In [24]:
grid['fire_label'] = (grid['fire_risk_score'] >= 0.6).astype(int)

In [26]:

print(grid['fire_label'].value_counts())

print(grid[['fire_risk_score','fire_label']].head())

fire_label
0    1217
1     408
Name: count, dtype: int64
   fire_risk_score  fire_label
0         0.733967           1
1         0.580929           0
2         0.502464           0
3         0.618346           1
4         0.494522           0


In [28]:
fire = grid[grid['fire_label']==1]

non_fire = grid[grid['fire_label']==0].sample(
    len(fire),
    random_state=42
)

data = pd.concat([fire, non_fire])

In [30]:
data.head()
non_fire.head()


,date,latitude,longitude,NDVI,soil_moisture,temperature,rainfall,elevation,slope,aspect,...,Rainfall,Wind_Speed,veg_dryness,temp_risk,humidity_risk,wind_risk,rain_risk,terrain_risk,fire_risk_score,fire_label
654,13-10-2023,10.90,79.84,0.342536,27.040123,24.182938,13.711835,2900.332896,13.676110,256.384520,...,6.758911,5.601064,0.657464,0.215190,0.815360,0.112021,0.864822,0.231254,0.488189,0
309,12-12-2023,23.32,77.40,0.381195,27.349709,13.813705,6.727501,631.873079,14.957534,125.715447,...,6.420431,12.426800,0.618805,0.845391,0.389327,0.248536,0.871591,0.252922,0.592474,0
56,15-02-2023,26.84,81.07,0.368845,27.486803,5.691288,6.617430,1836.109604,33.676632,315.842167,...,12.226348,2.962218,0.631155,0.801120,0.449990,0.059244,0.755473,0.569449,0.570031,0
1329,31-01-2023,26.76,81.02,0.546676,19.586415,4.302809,7.926993,2464.838143,32.146481,114.091922,...,1.121603,7.650035,0.453324,0.390600,0.621920,0.153001,0.977568,0.543575,0.474821,0
716,27-12-2023,29.30,76.43,0.515117,21.169768,12.006305,14.303584,383.249131,51.643835,268.736906,...,16.532836,13.804133,0.484883,0.250212,0.639954,0.276083,0.669343,0.873261,0.456021,0


In [32]:
grid['fire_probability'] = grid['fire_risk_score']

grid['fire_label'] = (
    np.random.rand(len(grid))
    < grid['fire_probability']
).astype(int)

In [34]:
data.head()
non_fire.head()

,date,latitude,longitude,NDVI,soil_moisture,temperature,rainfall,elevation,slope,aspect,...,Rainfall,Wind_Speed,veg_dryness,temp_risk,humidity_risk,wind_risk,rain_risk,terrain_risk,fire_risk_score,fire_label
654,13-10-2023,10.90,79.84,0.342536,27.040123,24.182938,13.711835,2900.332896,13.676110,256.384520,...,6.758911,5.601064,0.657464,0.215190,0.815360,0.112021,0.864822,0.231254,0.488189,0
309,12-12-2023,23.32,77.40,0.381195,27.349709,13.813705,6.727501,631.873079,14.957534,125.715447,...,6.420431,12.426800,0.618805,0.845391,0.389327,0.248536,0.871591,0.252922,0.592474,0
56,15-02-2023,26.84,81.07,0.368845,27.486803,5.691288,6.617430,1836.109604,33.676632,315.842167,...,12.226348,2.962218,0.631155,0.801120,0.449990,0.059244,0.755473,0.569449,0.570031,0
1329,31-01-2023,26.76,81.02,0.546676,19.586415,4.302809,7.926993,2464.838143,32.146481,114.091922,...,1.121603,7.650035,0.453324,0.390600,0.621920,0.153001,0.977568,0.543575,0.474821,0
716,27-12-2023,29.30,76.43,0.515117,21.169768,12.006305,14.303584,383.249131,51.643835,268.736906,...,16.532836,13.804133,0.484883,0.250212,0.639954,0.276083,0.669343,0.873261,0.456021,0


In [36]:
features = [
'NDVI',
'elevation',
'slope',
'aspect',
'Temperature_Avg',
'Humidity',
'Rainfall',
'Wind_Speed',
'veg_dryness',
'temp_risk',
'humidity_risk',
'wind_risk',
'rain_risk',
'terrain_risk',
'fire_risk_score'
]

X = data[features]

y = data['fire_label']

### SCALE FEATURES

In [39]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

pickle.dump(scaler, open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\scaler.pkl","wb"))

### TRAIN Random forest MODEL

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (652, 15)
Test shape: (164, 15)


In [42]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    random_state=42
)

rf_model.fit(X_train, y_train)

rf_pred = rf_model.predict_proba(X_test)[:,1]

In [43]:
import pickle

pickle.dump(rf_model, open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\rf_model.pkl","wb"))

### TRAIN XGBOOST

In [45]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [46]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=10,
    random_state=42
)

xgb_model.fit(X_train, y_train)

xgb_pred = xgb_model.predict_proba(X_test)[:,1]

In [53]:
pickle.dump(xgb_model, open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\xgb_model.pkl","wb"))

### TRAIN LSTM

In [66]:
pip install lightgbm


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.5 MB 991.0 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.5 MB 1.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 1.1 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.5 MB 1.5 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.5 MB 1.7 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.5 MB 1.9 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 1.8 MB/s eta 0:00:01
   ------------------- -------------------- 0.7/1.5 MB 2.1 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.5 MB 2.0 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.5 MB 2.3 MB/s eta 0:00:01
   ------------------------------ --------- 1.1/1.5 MB 2.3 MB/s eta 0:00:01
   ---------------------------------- ----- 1.2/1.5 MB 2.5 MB/s eta 0:00:01
   -------------

In [70]:
from lightgbm import LGBMClassifier
import pickle

lgb_model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=10,
    random_state=42
)

lgb_model.fit(X_train, y_train)

lgb_pred = lgb_model.predict_proba(X_test)[:,1]

pickle.dump(lgb_model, open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\lgb_model.pkl","wb"))

print("LightGBM trained")

[LightGBM] [Info] Number of positive: 326, number of negative: 326
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000129 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2726
[LightGBM] [Info] Number of data points in the train set: 652, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

In [75]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [77]:
from catboost import CatBoostClassifier

cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=10,
    verbose=0,
    random_state=42
)

cat_model.fit(X_train, y_train)

cat_pred = cat_model.predict_proba(X_test)[:,1]

pickle.dump(cat_model, open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\cat_model.pkl","wb"))

print("CatBoost trained")

CatBoost trained


In [81]:
import numpy as np
import pickle
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
ensemble_prob = (
    rf_pred +
    xgb_pred +
    lgb_pred +
    cat_pred
) / 4


final_pred = (ensemble_prob > 0.5).astype(int)


# ============================================
# 6. EVALUATION
# ============================================

accuracy = accuracy_score(y_test, final_pred)

print("\nFINAL ENSEMBLE ACCURACY:", accuracy)

print("\nClassification Report:\n",
      classification_report(y_test, final_pred))


# ============================================
# 7. SAVE ENSEMBLE FUNCTION
# ============================================

def ensemble_predict(sample):

    rf = rf_model.predict_proba(sample)[:,1]
    xgb = xgb_model.predict_proba(sample)[:,1]
    lgb = lgb_model.predict_proba(sample)[:,1]
    cat = cat_model.predict_proba(sample)[:,1]

    prob = (rf + xgb + lgb + cat) / 4

    return (prob > 0.5).astype(int), prob


print("\nEnsemble model ready.")


FINAL ENSEMBLE ACCURACY: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        82
           1       1.00      1.00      1.00        82

    accuracy                           1.00       164
   macro avg       1.00      1.00      1.00       164
weighted avg       1.00      1.00      1.00       164


Ensemble model ready.


In [83]:
import pickle
import numpy as np

# Load models
scaler = pickle.load(open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\scaler.pkl", "rb"))
rf = pickle.load(open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\rf_model.pkl", "rb"))
xgb = pickle.load(open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\xgb_model.pkl", "rb"))
lgb = pickle.load(open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\lgb_model.pkl", "rb"))
cat = pickle.load(open("C:\\Users\\hp\\Desktop\\Forestfire Project\\models\\cat_model.pkl", "rb"))

def predict_fire(input_data):

    input_scaled = scaler.transform(input_data)

    rf_pred = rf.predict_proba(input_scaled)[:,1]
    xgb_pred = xgb.predict_proba(input_scaled)[:,1]
    lgb_pred = lgb.predict_proba(input_scaled)[:,1]
    cat_pred = cat.predict_proba(input_scaled)[:,1]

    ensemble = (rf_pred + xgb_pred + lgb_pred + cat_pred) / 4

    return ensemble

In [87]:
import os

os.makedirs("dashboard", exist_ok=True)

with open("C:\\Users\\hp\\Desktop\\Forestfire Project\\app\\app.py", "w") as f:
    pass

print("app.py created successfully")

app.py created successfully
